# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_csv = "../output_data/cities.csv"
cities_data_csv = pd.read_csv(cities_data_csv)
cities_data_csv.drop('City_ID', inplace=True, axis=1)
cities_data_csv



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...
543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configuring gmaps
gmaps.configure(api_key=g_key)

locations = cities_data_csv[['Lat', 'Lng']]
humidity = cities_data_csv['Humidity']
max_humidity = humidity.max()

fig = gmaps.figure(center=(0,0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
perfect_weather = cities_data_csv.loc[(cities_data_csv['Wind Speed'] <= 10.5)][(cities_data_csv['Cloudiness'] == 0)][(cities_data_csv['Max Temp'] < 90)][(cities_data_csv['Max Temp'] > 35)][(cities_data_csv['Humidity'] > 40)][(cities_data_csv['Humidity'] < 50)].dropna()
perfect_weather

C:\Users\kazek\AppData\Local\Temp\ipykernel_17940\922101608.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perfect_weather = cities_data_csv.loc[(cities_data_csv['Wind Speed'] <= 10.5)][(cities_data_csv['Cloudiness'] == 0)][(cities_data_csv['Max Temp'] < 90)][(cities_data_csv['Max Temp'] > 35)][(cities_data_csv['Humidity'] > 40)][(cities_data_csv['Humidity'] < 50)].dropna()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
263,markova,0,RU,1558378835,49,52.21,104.21,46.40,6.71
314,baruun-urt,0,MN,1558378847,48,46.68,113.28,44.88,5.32
346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
371,carballo,0,ES,1558378860,44,43.21,-8.69,63.00,5.82
518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72
533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perfect_weather
hotel_df['Hotel Name'] = ''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
263,markova,0,RU,1558378835,49,52.21,104.21,46.40,6.71,
314,baruun-urt,0,MN,1558378847,48,46.68,113.28,44.88,5.32,
346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,
371,carballo,0,ES,1558378860,44,43.21,-8.69,63.00,5.82,
518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72,
533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82,


In [6]:
# reindexing dataframe
hotel_df = hotel_df.reset_index()
del hotel_df['index']
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,markova,0,RU,1558378835,49,52.21,104.21,46.40,6.71,
1,baruun-urt,0,MN,1558378847,48,46.68,113.28,44.88,5.32,
2,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78,
3,carballo,0,ES,1558378860,44,43.21,-8.69,63.00,5.82,
4,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72,
5,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82,


In [10]:
params = {
     'radius': 5000,
     'types': 'lodging',
     'key': g_key,
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    city_name = row['City']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    params['location'] = f'{lat},{lon}'
    print(f'Retrieving Results for Index {index} - {city_name}')
    response = requests.get(base_url, params=params)
    response = response.json()

    try:
     
     print(f'Nearest hotel in {city_name} is {response["results"][0]["name"]}.')
     hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
     print('------------------------------')

    except:
          print('Empty field')
     

Retrieving Results for Index 0 - markova
Nearest hotel in markova is Elderly "Sosnovy Bor".
------------------------------
Retrieving Results for Index 1 - baruun-urt
Nearest hotel in baruun-urt is Ахмадын амралт сувилал.
------------------------------
Retrieving Results for Index 2 - chiredzi
Nearest hotel in chiredzi is Clevers Hotel Chiredzi.
------------------------------
Retrieving Results for Index 3 - carballo
Nearest hotel in carballo is Pazo do Souto.
------------------------------
Retrieving Results for Index 4 - bonito
Nearest hotel in bonito is Marruá Hotel.
------------------------------
Retrieving Results for Index 5 - hakkari
Nearest hotel in hakkari is Vaihmalan Hovi.
------------------------------


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))